In [1]:
import pandas as pd
import numpy as np


In [2]:
URLS = [
    'http://eurosvet.ru/prices/prices-eurosvet-rur-utf-8.csv',
    'http://elektrostandard.ru/prices/prices-elektrostandard-rur-utf-8.csv',
    'http://werkel.ru/prices/prices-werkel-rur-utf-8.csv',
    'https://api.escorp.ru:779/prices/prices-minimir-rur-utf-8.csv'
]
dataframes = []
for url in URLS:
    dataframes.append(pd.read_csv(url, sep=';', encoding='utf-8'))


In [3]:
# make shop dataset
catalogue_product_headers = ['id', 'price',
                             'description', 'title', 'name', 'upc']
catalogue_product = pd.DataFrame([], columns=catalogue_product_headers)


In [4]:
def merge_columns(dataframes, col_name,):
    merged_column = []
    for dataframe in dataframes:
        merged_column.append(dataframe[col_name].to_numpy())
    return np.concatenate(merged_column)


In [5]:
what_copy = {
    'price': 'Цена',
    'description': 'Описание',
    'title': 'SEO название для сайта',
    'name': 'Наименование',
    'upc': 'Штрихкод'
    # 'slug':'SEO название',
}
for key in what_copy:
    catalogue_product[key] = merge_columns(dataframes, what_copy[key])
catalogue_product['id'] = np.arange(1, len(catalogue_product['id'])+1)
# shop_data['name']=merge_columns(dataframes,'Наименование')


In [6]:
# catalogue_product

In [7]:
# shop_data.to_csv('saved_shop.csv',index=False)

In [8]:
catalogue_productimage_headers = [
    'id', 'original', 'display_order', 'product_id']
catalogue_productimage = pd.DataFrame(
    [], columns=catalogue_productimage_headers)


In [16]:
def isNaN(num):
    return num != num


def make_list_of_images(row):
    merged_urls = []
    for i in range(len(row)):
        image_head = 'Изображение '+str(i)
        if image_head in row.keys() and not isNaN(row[image_head]):
            merged_urls.append(row[image_head])
    return merged_urls


def make_image_rows(to_dataframe: pd.DataFrame, from_dataframe):
    for i in range(len(from_dataframe.index)):
        image_urls = make_list_of_images(from_dataframe.iloc[i])
        for t in range(len(image_urls)):
            id = len(to_dataframe.index)+1
            to_dataframe.loc[id] = [id, image_urls[t], t, i]
    return to_dataframe


def fill_images_table(to_dataframe: pd.DataFrame, from_dataframes):
    for from_dataframe in from_dataframes:
        to_dataframe.merge(make_image_rows(to_dataframe, from_dataframe))
    return to_dataframe


catalogue_productimage = fill_images_table(catalogue_productimage, dataframes)


In [19]:
catalogue_productimage.to_excel('saved_shop.xlsx',index=False)
